# SBP Reader - Locosys SiRF Binary (Packed)

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys

from datetime import datetime, timedelta, timezone

import numpy as np

import unittest

## Main Class

In [2]:
class SbpReader():
    '''SBP file - Locosys SiRF Binary (Packed)'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        self.filename = filename


    def load(self):
        '''Load file into memory'''

        with open(self.filename, "rb") as f:
            self.readHeader(f)
            self.readData(f)
            self.consumeData()


    def readHeader(self, f):
        '''Load header into memory'''

        self.header = {}

        header = f.read(64)

        # External length excludes itself but includes the start token (a0a2) + checksum + end token (b0b3)
        length = header[1] << 8 | header[0]
        if length > 62:
            raise ValueError('Invalid length {}'.format(length))

        # Check the start token is a0a2
        startToken = header[2] << 8 | header[3]
        if startToken != 0xa0a2:
            raise ValueError('Unexpected start token {:x} - expected {:x}'.format(startToken, 0xa0a2))

        # Internal length is sometimes big endian, sometimes little endian!
        if header[4] == 0:
            lengthAlt = header[5]
        else:
            lengthAlt = header[4]

        # Internal length excludes the start token (a0a2) + itself + checksum + end token (b0b3)
        if lengthAlt != length - 8:
            raise ValueError('Unexpected internal length {} (external length {})'.format(lengthAlt, length))

        # Main content
        content = header[6 : length - 2]

        # Parse ASCII data, ignoring content[0] which is the mid-file ID
        splitContent = content[1:].decode('ascii').split(',')
        self.header['username'] = splitContent[0]
        self.header['serial'] = int(splitContent[1])
        self.header['frequency'] = 5 if splitContent[2] == '2' else 1
        self.header['firmware'] = splitContent[3]

        # Evaluate checksum
        checksum = header[length - 2] << 8 | header[length - 1]
        if checksum != sum(content):
            raise ValueError('Checksum difference - {}'.format(checksum - sum(content)))

        # Check the end token is b0b3
        endToken = header[length] << 8 | header[length + 1]
        if endToken != 0xb0b3:
            raise ValueError('Unexpected end token {:x} - expected {:x}'.format(endToken, 0xb0b3))


    def readData(self, f):
        '''Read data into memory'''

        fileSize = os.path.getsize(self.filename)

        if fileSize % 32 != 0:
            raise ValueError('File is not a multiple of 32 bytes')

        self.numRecords = (fileSize - 64) // 32

        # Note: The original SBP format had "bit flags" + "reserved" (GT-11), prior to "sdop" + "vsdop" (GT-31 onwards)
        dataType = np.dtype([
            ('hdop', 'u1'),
            ('sv_count', 'u1'),
            ('utc_secs', '<u2'),
            ('utc_packed_datetime', '<u4'),
            ('sv_ids', '<u4'),
            ('lat', '<i4'),
            ('lon', '<i4'),
            ('alt', '<i4'),
            ('sog', '<u2'),
            ('cog', '<u2'),
            ('roc', '<i2'),
            ('sdop', 'u1'),
            ('vsdop', 'u1')
        ])

        self.rawData = np.fromfile(f, dtype=dataType, count=self.numRecords)


    def consumeData(self):
        '''Consume data - convert to standard data structure'''

        self.data = {}

        # Convert HDOP from integer to decimal
        self.data['hdop'] = self.rawData['hdop'] / 5

        # Satellite count is already an integer
        self.data['sat'] = self.rawData['sv_count']

        # Convert time to milliseconds
        secs = self.rawData['utc_secs'] // 1000

        # Convert packed date/time to regular timestamp (seconds)
        vfunc = np.vectorize(decodePackedDateTime)
        timestampSecs = vfunc(self.rawData['utc_packed_datetime'])

        # Check that seconds and timestamps are consistent
        maxDiff = np.abs(secs - timestampSecs % 60).max()
        if maxDiff > 0:
            raise ValueError('Mismatch of seconds and timestamps')

        # Convert timestamps to milliseconds
        self.data['timestamp'] = timestampSecs * 1000 + self.rawData['utc_secs'] % 1000

        # Copy satellite IDs
        self.data['sv_ids'] = self.rawData['sv_ids']

        # Convert latitude and longitude to decimal
        self.data['lat'] = self.rawData['lat'] / 10000000
        self.data['lon'] = self.rawData['lon'] / 10000000

        # Convert elevation and SOG from cm/s to m/s
        self.data['ele'] = self.rawData['alt'] / 100
        self.data['sog'] = self.rawData['sog'] / 100

        # Convert COG from integer to decimal
        self.data['cog'] = self.rawData['cog'] / 100

        # Convert rate of climb from cm/s to m/s
        self.data['roc'] = self.rawData['roc'] / 100

        # Convert SDOP and VSDOP from cm/s to m/s
        self.data['sdop'] = self.rawData['sdop'] / 100
        self.data['vsdop'] = self.rawData['vsdop'] / 100

        # TODO - support SBP files with "bit flags" + "reserved" (GT-11) instead of "sdop" + "vsdop" (GT-31 onwards)


def decodePackedDateTime(packedDateTime):
    '''Decode packed date/time'''

    second = packedDateTime & 0x3f      
    packedDateTime >>= 6

    minute = packedDateTime & 0x3f
    packedDateTime >>= 6

    hour = packedDateTime & 0x1f
    packedDateTime >>= 5

    day = packedDateTime & 0x1f
    packedDateTime >>= 5

    month = packedDateTime % 12
    year = packedDateTime // 12 + 2000

    dt = datetime(year, month, day, hour, minute, second)
    timestamp = int((dt - datetime(1970, 1, 1)) / timedelta(seconds=1))

    return timestamp

## Unit Tests

In [3]:
class TestHeader(unittest.TestCase):
    '''Class to test header was correctly loaded'''

    def testUsername(self):
        '''Test the username is as expected'''

        self.assertEqual(sbpReader.header['username'], 'K888')


    def testSerial(self):
        '''Test the serial is as expected'''

        self.assertEqual(sbpReader.header['serial'], 155800017)


    def testFrequency(self):
        '''Test the frequency is as expected'''

        self.assertEqual(sbpReader.header['frequency'], 1)


    def testFirmware(self):
        '''Test the firmware is as expected'''

        self.assertEqual(sbpReader.header['firmware'], 'V1.2G0529C')

In [4]:
class TestData(unittest.TestCase):
    '''Class to test data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(sbpReader.numRecords, 9319)

        for fieldName in sbpReader.data:
            self.assertEqual(sbpReader.data[fieldName].size, 9319)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(sbpReader.data['hdop'].min(), 0.8)
        self.assertEqual(sbpReader.data['hdop'].max(), 3.4)


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(sbpReader.data['sat'].min(), 3)
        self.assertEqual(sbpReader.data['sat'].max(), 10)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(sbpReader.data['timestamp'].min(), 1649594705000)
        self.assertEqual(sbpReader.data['timestamp'].max(), 1649604023000)


    def testSatelliteIds(self):
        '''Test the satellite IDs are as expected'''

        self.assertEqual(sbpReader.data['sv_ids'].min(), 100827840)
        self.assertEqual(sbpReader.data['sv_ids'].max(), 1460305944)


    def testLatitude(self):
        '''Test the latitude is as expected'''

        self.assertEqual(sbpReader.data['lat'].min(), 50.5708492)
        self.assertEqual(sbpReader.data['lat'].max(), 50.5931109)


    def testLongitude(self):
        '''Test the longitude is as expected'''

        self.assertEqual(sbpReader.data['lon'].min(), -2.4620382)
        self.assertEqual(sbpReader.data['lon'].max(), -2.4427624)


    def testElevation(self):
        '''Test the elevation is as expected'''

        self.assertEqual(sbpReader.data['ele'].min(), -16.45)
        self.assertEqual(sbpReader.data['ele'].max(), 100.88)


    def testSpeed(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(sbpReader.data['sog'].min(), 0)
        self.assertEqual(sbpReader.data['sog'].max(), 12.70)


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(sbpReader.data['cog'].min(), 0)
        self.assertEqual(sbpReader.data['cog'].max(), 359.8)


    def testRoc(self):
        '''Test the rate of climb is as expected'''

        self.assertEqual(sbpReader.data['roc'].min(), -0.48)
        self.assertEqual(sbpReader.data['roc'].max(), 0.4)


    def testSdop(self):
        '''Test the speed dilution of precision is as expected'''

        self.assertEqual(sbpReader.data['sdop'].min(), 0.01)
        self.assertEqual(sbpReader.data['sdop'].max(), 2.55)


    def testVsdop(self):
        '''Test the vertical speed dilution of precision is as expected'''

        self.assertEqual(sbpReader.data['vsdop'].min(), 0.01)
        self.assertEqual(sbpReader.data['vsdop'].max(), 2.55)

In [5]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220410', 'GW52_1Hz_K888_155800017_20220410_230017.sbp')

    sbpReader = SbpReader(filename)
    sbpReader.load()

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

.................
----------------------------------------------------------------------
Ran 17 tests in 0.009s

OK
